In [1]:
import os
import re
import psycopg2
import requests
import json, urllib
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [23]:
def back_to_list(df, col):
    """Convert columns in df back to lists from strings (this happens when saving to csv)
    """
    
    df[col] = [s.strip("'[") for s in df[col]]
    df[col] = [s.strip("]'") for s in df[col]]
    df[col] = [s.split("', '") for s in df[col]]
    
    return df[col]

In [2]:
# Load education and occupation details

jobs_df = pd.read_csv('/Users/amanda/Documents/Projects/insight/data/processed/jobbank-fields.csv')
details_df = pd.read_csv('/Users/amanda/Documents/Projects/insight/data/processed/jobbank-details.csv')
job_name_df = pd.read_csv('/Users/amanda/Documents/Projects/insight/data/processed/education-to-job.csv')


# Clean and organize skills information

In [25]:
jobs_df['description'].str.split('found under ')
edu_categories = [s.split('found under ')[1] for s in jobs_df['description']]
edu_categories = [s.strip('.') for s in edu_categories]
jobs_df['education-groups'] = edu_categories

jobs_df['top-jobs'] = back_to_list(jobs_df, 'top-jobs')
jobs_df['top-job-links'] = back_to_list(jobs_df, 'top-job-links')
jobs_df['job-percent'] = back_to_list(jobs_df, 'job-percent')

education_df = jobs_df[['degree','education-groups','top-jobs','job-percent']].drop_duplicates(subset =['education-groups','degree']) 

In [26]:
details_df['requirements'] = back_to_list(details_df, 'requirements')
details_df['skills'] = back_to_list(details_df, 'skills')

# Build dataframe for Sankey diagram

In [5]:
job_name_df = pd.read_csv('/Users/amanda/Documents/Projects/insight/data/processed/education-to-job.csv')

#job_name_df['identifier'] = job_name_df['education-groups'].str.cat(job_name_df['degree'],sep=" : ")

#job_name_df['job-percent'].fillna('', inplace=True)

#job_name_df['job-percent'] = [s.strip('%') for s in job_name_df['job-percent']]
#job_name_df['job-percent'] = pd.to_numeric(job_name_df['job-percent'], errors='coerce')

In [6]:
job_name_df

,degree,education-groups,top-jobs,job-percent,top-job-links,identifier
0,College/CEGEP,Accounting,Accounting and related clerks,11.99,/marketreport/summary-occupation/14122/ca;jses...,Accounting : College/CEGEP
1,College/CEGEP,Accounting,Financial auditors and accountants,11.23,/marketreport/summary-occupation/131/ca;jsessi...,Accounting : College/CEGEP
2,College/CEGEP,Accounting,Accounting technicians and bookkeepers,10.46,/marketreport/summary-occupation/24500/ca;jses...,Accounting : College/CEGEP
3,College/CEGEP,Accounting,Administrative officers,3.92,/marketreport/summary-occupation/12462/ca;jses...,Accounting : College/CEGEP
4,College/CEGEP,Accounting,Retail salespersons,3.25,/marketreport/summary-occupation/20599/ca;jses...,Accounting : College/CEGEP
...,...,...,...,...,...,...
1940,Master's degree,Sociology,Administrative officers,2.63,/marketreport/summary-occupation/12462/ca;jses...,Sociology : Master's degree
1941,Master's degree,Sociology,Social and community service workers,2.63,/marketreport/summary-occupation/5112/ca;jsess...,Sociology : Master's degree
1942,Master's degree,Sociology,Business development officers and marketing re...,2.63,/marketreport/summary-occupation/3916/ca;jsess...,Sociology : Master's degree
1943,Master's degree,Sociology,University professors and lecturers,1.97,/marketreport/summary-occupation/4707/ca;jsess...,Sociology : Master's degree


In [29]:
def build_sankey(df, degree):

    data = df.where(df['education-groups']==degree).dropna()
    
    

    data_trace = dict(type='sankey',
                      domain = dict(x =  [0,1],y =  [0,1]),
                      orientation = "h",
                      valueformat = ".0f",
                      node = dict(pad = 10,
                                  thickness = 30,
                                  line = dict(color = "black",width = 0),
                                  label =  ["A1", "A2", "B1", "B2", "C1", "C2"],
                                  #color = scottish_df['Color']
                                 ),
                      link = dict(source = [0, 1, 0, 2, 3, 3],
                                  target = [2, 3, 3, 4, 4, 5],
                                  value = [8, 4, 2, 8, 4, 2],
                                  #color = scottish_df['Link Color'].dropna(axis=0, how='any'),
                                 )
                     )

    layout =  dict(title = "Title",
                   height = 772,
                   font = dict(size = 10),
                  )

    fig = go.Figure(data=[data_trace], layout=layout)

    return fig

In [12]:
def build_sankey(df, degree):

    data = df.where(df['education-groups']==degree).dropna()
    
    all_nodes = data['identifier'].values.tolist() + data['top-jobs'].values.tolist()
    source_indices = [all_nodes.index(identifier) for identifier in data['identifier']]
    target_indices = [all_nodes.index(top_job) for top_job in data['top-jobs']]

    fig = go.Figure(data=[go.Sankey
                          (node = dict(pad = 15,
                                       thickness = 20,
                                       line = dict(color = "black", 
                                                   width = 0),
                                       label = all_nodes,
                           ),
                           link = dict(source = source_indices, # indices correspond to labels, eg A1, A2, A2, B1, ...
                                       target = target_indices,
                                       value = data['job-percent']*100)
                          )
                         ]
                   )

    fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
    fig.show()
    
    return data
    

In [13]:
data = build_sankey(job_name_df,'Accounting')

In [82]:
job_name_df.where(job_name_df['education-groups']=='Accounting').dropna()

,degree,education-groups,top-jobs,job-percent,identifier
0,College/CEGEP,Accounting,Accounting and related clerks,11.99%,Accounting : College/CEGEP
1,College/CEGEP,Accounting,Financial auditors and accountants,11.23%,Accounting : College/CEGEP
2,College/CEGEP,Accounting,Accounting technicians and bookkeepers,10.46%,Accounting : College/CEGEP
3,College/CEGEP,Accounting,Administrative officers,3.92%,Accounting : College/CEGEP
4,College/CEGEP,Accounting,Retail salespersons,3.25%,Accounting : College/CEGEP
5,College/CEGEP,Accounting,Administrative assistants,3.06%,Accounting : College/CEGEP
6,College/CEGEP,Accounting,Retail and wholesale trade managers,2.63%,Accounting : College/CEGEP
7,College/CEGEP,Accounting,General office support workers,2.53%,Accounting : College/CEGEP
8,College/CEGEP,Accounting,Other customer and information services repres...,2.48%,Accounting : College/CEGEP
9,College/CEGEP,Accounting,Customer services representatives - financial ...,2.05%,Accounting : College/CEGEP


In [108]:
nz_migration = pd.read_csv("~/Downloads/migration_nz.csv")

nz_migration = nz_migration[nz_migration["Measure"]!="Net"]
nz_migration = nz_migration[~nz_migration["Country"].isin(["Not stated", "All countries"])]
nz_migration_grouped = nz_migration.groupby(by=["Measure","Country"]).sum()[["Value"]]
nz_migration_grouped = nz_migration_grouped.reset_index()
nz_migration_grouped.head()

,Measure,Country,Value
0,Arrivals,Afghanistan,1644.0
1,Arrivals,Africa and the Middle East,149784.0
2,Arrivals,Albania,178.0
3,Arrivals,Algeria,143.0
4,Arrivals,American Samoa,2412.0


In [113]:
continents = ["Asia", "Australia","Africa and the Middle East","Europe", "Americas", "Oceania"]
continent_wise_migration = nz_migration_grouped[nz_migration_grouped.Country.isin(continents)]
continent_wise_migration

,Measure,Country,Value
1,Arrivals,Africa and the Middle East,149784.0
5,Arrivals,Americas,267137.0
14,Arrivals,Asia,795697.0
15,Arrivals,Australia,1057127.0
74,Arrivals,Europe,1044693.0
166,Arrivals,Oceania,1331987.0
252,Departures,Africa and the Middle East,63555.0
256,Departures,Americas,245915.0
265,Departures,Asia,317603.0
266,Departures,Australia,2325398.0


In [111]:
all_nodes = continent_wise_migration.Measure.values.tolist() + continent_wise_migration.Country.values.tolist()
source_indices = [all_nodes.index(measure) for measure in continent_wise_migration.Measure]
target_indices = [all_nodes.index(country) for country in continent_wise_migration.Country]

fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      label =  all_nodes,
      color =  "red"
    ),

    # Add links
    link = dict(
      source =  source_indices,
      target =  target_indices,
      value =  continent_wise_migration.Value,
))])

fig.update_layout(title_text="Population Migration between New Zealand and Other Continents",
                  font_size=10)
fig.show()

In [129]:
rng = pd.date_range('1/1/2011', periods=7500, freq='H')
ts = pd.Series(np.random.randn(len(rng)), index=rng)

graphs = [
    dict(
        data=[
            dict(
                x=[1, 2, 3],
                y=[10, 20, 30],
                type='scatter'
            ),
        ],
        layout=dict(
            title='first graph'
        )
    ),

    dict(
        data=[
            dict(
                x=[1, 3, 5],
                y=[10, 50, 30],
                type='bar'
            ),
        ],
        layout=dict(
            title='second graph'
        )
    ),

    dict(
        data=[
            dict(
                x=ts.index,  # Can use the pandas data structures directly
                y=ts
            )
        ]
    )
]



In [130]:
graphs

[{'data': [{'x': [1, 2, 3], 'y': [10, 20, 30], 'type': 'scatter'}],
  'layout': {'title': 'first graph'}},
 {'data': [{'x': [1, 3, 5], 'y': [10, 50, 30], 'type': 'bar'}],
  'layout': {'title': 'second graph'}},
 {'data': [{'x': DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
                   '2011-01-01 02:00:00', '2011-01-01 03:00:00',
                   '2011-01-01 04:00:00', '2011-01-01 05:00:00',
                   '2011-01-01 06:00:00', '2011-01-01 07:00:00',
                   '2011-01-01 08:00:00', '2011-01-01 09:00:00',
                   ...
                   '2011-11-09 02:00:00', '2011-11-09 03:00:00',
                   '2011-11-09 04:00:00', '2011-11-09 05:00:00',
                   '2011-11-09 06:00:00', '2011-11-09 07:00:00',
                   '2011-11-09 08:00:00', '2011-11-09 09:00:00',
                   '2011-11-09 10:00:00', '2011-11-09 11:00:00'],
                  dtype='datetime64[ns]', length=7500, freq='H'),
    'y': 2011-01-01 00:00:00   -1.34

In [7]:
job_name_df.to_csv('/Users/amanda/Documents/Projects/insight/data/processed/education-to-job.csv', index=False)